### Modelo Supervisionado - KNN

O modelo KNN (K-Vizinhos Mais Próximos) é um algoritmo de aprendizado supervisionado que classifica ou prevê um novo ponto de dados com base nos seus 'k' vizinhos mais próximos, assumindo que pontos similares tendem a se agrupar.

**Objetivo do Modelo**: <br>
Identificar desempenho de cada categoria nas localidades de novas lojas previstas

**Datasets Utilizados**: <br>
- dataset_limpo.csv: dataset com as informações de venda das lojas;

- sugestoes_expansao_para_supervisionado.csv: dataset com as informações das novas localidades identificadas (previstas) para abertura de novas lojas (sem dados prévios de venda);

- com coordenadas.csv: dataset com as coordenadas de localização de todas as lojas (para fazer uma análise específica, não apenas por grandes regiões como norte, sul, etc...).


### Etapa 1 - Importação de bibliotecas e dos dados 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Importação de bases
df_vendas = pd.read_csv('../../database/dataset gerado/dataset_limpo.csv')
df_sugestoes = pd.read_csv('../../database/dataset gerado/sugestoes_expansao_para_supervisionado.csv')
df_coords = pd.read_csv('../../database/dataset gerado/com_coordenadas.csv')

# Criando categorias mescladas (hierarquia Grupo_Produto + GRUPO_CHILLI)
def norm_cat(s):
    return (s.astype(str)
        .str.strip()
        .str.upper()
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace(r'\s+', '_', regex=True))

df_vendas['Categoria_Combo'] = (
    norm_cat(df_vendas['Dim_Produtos.Grupo_Produto']).fillna('DESCONHECIDO') + '_' + 
    norm_cat(df_vendas['Dim_Produtos.GRUPO_CHILLI']).fillna('DESCONHECIDO')
)

# Conferindo tamanho
print(f"Tamanho df_vendas: {df_vendas.shape}")
print(f"Tamanho df_sugestoes: {df_sugestoes.shape}")
print(f"Tamanho df_coords: {df_coords.shape}")

# Olhando as primeiras linhas
display(df_vendas.head())
display(df_sugestoes.head())
display(df_coords.head())

print('Categorias únicas (Grupo_Produto):', df_vendas['Dim_Produtos.Grupo_Produto'].nunique())
print('Categorias únicas (GRUPO_CHILLI):', df_vendas['Dim_Produtos.GRUPO_CHILLI'].nunique())
print('Categorias únicas (Categoria_Combo):', df_vendas['Categoria_Combo'].nunique())


### Etapa 2 - Padronizar texto das localizações e mergear coordenadas

In [ ]:
# Padronização de texto 
def padroniza_texto(s: pd.Series) -> pd.Series:
    s = s.copy()
    mask = s.notna()  # só transforma onde NÃO é nulo
    s.loc[mask] = (s.loc[mask].astype(str)
                   .str.strip()
                   .str.upper()               
                   .str.normalize('NFKD')     
                   .str.encode('ascii', errors='ignore')
                   .str.decode('utf-8'))
    return s

chave3 = ["Dim_Lojas.Bairro_Emp", "Dim_Lojas.Cidade_Emp", "Dim_Lojas.Estado_Emp"]

# Padronizar chaves em AMBAS as tabelas
for col in chave3:
    df_vendas[col] = padroniza_texto(df_vendas[col])
    df_coords[col] = padroniza_texto(df_coords[col])

# Tornar df_coords ÚNICO por (Bairro, Cidade, Estado)
# usar média = centróide quando houver múltiplos registros da mesma chave
df_coords_uniq = (
    df_coords
      .groupby(chave3, as_index=False)[["Latitude", "Longitude"]]
      .mean()
)

# Checagem de unicidade após agregação
assert not df_coords_uniq.duplicated(subset=chave3).any(), "df_coords_uniq ainda tem duplicatas na chave."

# MERGE LIMPO em um NOVO DataFrame + validação de cardinalidade
df_vendas_geo = df_vendas.merge(
    df_coords_uniq[chave3 + ["Latitude", "Longitude"]],
    on=chave3,
    how="left",
    validate="many_to_one",   
    indicator=True
)

print("Linhas base (df_vendas):", len(df_vendas))
print("Linhas pós-merge (df_vendas_geo):", len(df_vendas_geo))
print("Nulos em Latitude:", df_vendas_geo["Latitude"].isna().sum())
print(df_vendas_geo["_merge"].value_counts())

# usar df_vendas_geo como base corrente
df_vendas = df_vendas_geo.drop(columns=["_merge"])

# visualização incial da base
display(df_vendas.head())

# Análise dos faltantes
df_vendas['Latitude']  = pd.to_numeric(df_vendas['Latitude'], errors='coerce')
df_vendas['Longitude'] = pd.to_numeric(df_vendas['Longitude'], errors='coerce')

print("Nulos em Latitude:", df_vendas['Latitude'].isna().sum())
print("Nulos em Longitude:", df_vendas['Longitude'].isna().sum())

faltantes = df_vendas[df_vendas['Latitude'].isna()]
print("Total faltantes:", len(faltantes))
display(faltantes.head())

# Agrupa e conta faltantes por cidade/estado
mapa_faltantes = (
    faltantes
      .groupby(["Dim_Lojas.Cidade_Emp","Dim_Lojas.Estado_Emp"])
      .size()
      .reset_index(name="qtd_faltantes")
      .sort_values("qtd_faltantes", ascending=False)
)

display(mapa_faltantes.head(20))

# Agrupa e conta faltantes por categoria
faltantes_cat = (
    faltantes['Categoria_Combo']
      .value_counts()
      .reset_index()
      .rename(columns={'index':'Categoria_Combo','Categoria_Combo':'qtd'})
)

display(faltantes_cat.head(10))

# Remover registros sem latitude ou longitude
df_vendas = df_vendas.dropna(subset=["Latitude", "Longitude"]).reset_index(drop=True)

# Conferindo resultado
print("Shape após drop:", df_vendas.shape)
print("Nulos em Latitude:", df_vendas['Latitude'].isna().sum())
print("Nulos em Longitude:", df_vendas['Longitude'].isna().sum())




#### Descarte de registros sem coordenadas

Durante a integração entre a base de vendas (`df_vendas`) e a base de coordenadas (`df_coords`), foram identificados **189 registros (≈0,6% do total)** sem informações de latitude/longitude. Esses casos estavam concentrados em apenas **9 cidades específicas** e distribuídos principalmente entre as categorias **ÓCULOS** e **ACESSÓRIOS**.

Decidimos **dropar esses registros** porque:
- representam uma fração mínima da base, com impacto estatístico irrelevante;
- manter valores nulos exigiria tratamentos adicionais sem ganho real para o modelo;
- garantir uma base **100% consistente geograficamente** simplifica as próximas etapas do pipeline (rotulagem e treinamento do KNN).

Após o descarte, seguimos com **31.217 registros válidos**, assegurando consistência espacial sem comprometer a robustez da análise.


### Etapa 3 - Gerar os rótulos (bom / mediano / ruim) por localidade × Categoria_Combo usando o volume de vendas.

In [ ]:
col_cidade = "Dim_Lojas.Cidade_Emp"
col_uf     = "Dim_Lojas.Estado_Emp"

# Cria localidade = "CIDADE-UF"
df_vendas["localidade"] = df_vendas[col_cidade].astype(str).str.strip() + "-" + df_vendas[col_uf].astype(str).str.strip()

# Sanity check
print("Localidades únicas:", df_vendas["localidade"].nunique())
print("\nTop 10 localidades por nº de linhas:")
display(df_vendas["localidade"].value_counts().head(10))

# Agregar volume de vendas por localidade e categoria
df_volumes = (
    df_vendas
      .groupby(["localidade", "Categoria_Combo"])
      .size() 
      .reset_index(name="volume_vendas")
)

print("Shape da tabela agregada:", df_volumes.shape)
display(df_volumes.head(10))

def rotular_categorias(grupo):

    grupo = grupo.sort_values(by='volume_vendas', ascending=False).reset_index(drop=True)
    n = len(grupo)

    if n == 1:
        grupo["desempenho"] = "bom"
    elif n == 2:
        grupo["desempenho"] = ["bom", "ruim"]
    else:
        tercil_1 = int(n / 3)
        tercil_2 = int(2 * n / 3)

        grupo.loc[:tercil_1 - 1, "desempenho"] = "bom"
        grupo.loc[tercil_1+1: tercil_2, "desempenho"] = "mediano"
        grupo.loc[tercil_2 + 1:, "desempenho"] = "ruim"
    
    return grupo

df_rotulado = df_volumes.groupby("localidade", group_keys=False).apply(rotular_categorias)

print("Shape da tabela rotulada:", df_rotulado.shape)
display(df_rotulado.head(10))

print("Visualização da distribuição geral dos rótulos:")

# Contagem geral dos rótulos
label_counts = df_rotulado["desempenho"].value_counts()

plt.figure(figsize=(6,4))
label_counts.plot(kind="bar", color=["#2ca02c","#ff7f0e","#d62728"])  # verde, laranja, vermelho
plt.title("Distribuição geral dos rótulos de desempenho")
plt.xlabel("Rótulo")
plt.ylabel("Nº de pares localidade × categoria")
plt.show()

print(label_counts)



### Observação sobre a agregação (`localidade × Categoria_Combo`)

Após a agregação, obtivemos **1.540 pares distintos de `localidade × Categoria_Combo`**.  
Isso significa que, no período, **cada cidade não teve vendas em todas as 17 categorias possíveis**.  

- Em localidades **menores**, aparecem apenas 1 ou 2 categorias.  
- Em localidades **maiores**, espera-se encontrar um número maior de categorias ativas.  

Esse comportamento é esperado e reflete a realidade de vendas:  
cada mercado local concentra-se apenas em parte do portfólio da marca.  


### Etapa 4 - Separar a base em conjunto de teste e treino

In [ ]:
# =========================
# ETAPA 4 — Base de modelagem com LOCALIDADE como feature
# =========================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 4.1) Ligar df_rotulado (localidade × Categoria_Combo × desempenho) às REGIÕES por localidade

def moda(s: pd.Series):
    m = s.mode(dropna=True)
    return m.iloc[0] if not m.empty else np.nan

# Tabela de atributos regionais por localidade (valor mais frequente)
regioes_por_localidade = (
    df_vendas.groupby("localidade")[["Dim_Lojas.Regiao", "Dim_Lojas.REGIAO_CHILLI"]]
             .agg(moda)
             .reset_index()
)

# Base final para modelagem: rótulo + regiões (mantém localidade explícita)
df_modelo = (
    df_rotulado.merge(regioes_por_localidade, on="localidade", how="left")
)

print("df_modelo shape:", df_modelo.shape)
display(df_modelo.head())

# 4.2) Preparar X e y (SEM vazamento: NÃO usar volume_vendas)
y = df_modelo["desempenho"].copy()

# >>> AQUI entra a granularidade: 'localidade' como feature categórica
X = df_modelo[["localidade", "Categoria_Combo", "Dim_Lojas.Regiao", "Dim_Lojas.REGIAO_CHILLI"]].copy()

# Remover linhas com rótulo nulo (se houver)
mask = y.notna()
X, y = X[mask], y[mask]

# One-hot encoding das categóricas (inclui LOCALIDADE)
X = pd.get_dummies(
    X,
    columns=["localidade", "Categoria_Combo", "Dim_Lojas.Regiao", "Dim_Lojas.REGIAO_CHILLI"],
    drop_first=True
)

print("Shape de X após one-hot:", X.shape)
print("Classes de y:", y.unique())

# 4.3) Split treino/teste estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("X_train:", X_train.shape, "| X_test:", X_test.shape)
print("\nDistribuição rótulos treino (%):")
print((y_train.value_counts(normalize=True) * 100).round(2))

print("\nDistribuição rótulos teste (%):")
print((y_test.value_counts(normalize=True) * 100).round(2))


### Etapa 5 - Treinar e avaliar modelo (acurácia, matriz de confusão e classification report)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt

# Testar k de 1 até 30
k_values = range(1, 31)
cv_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring="accuracy")
    cv_scores.append(scores.mean())

best_k = k_values[np.argmax(cv_scores)]
best_score = max(cv_scores)

print(f"Melhor k encontrado: {best_k} com acurácia média de {best_score:.2%}")

# Plot da curva de validação
plt.figure(figsize=(8,5))
plt.plot(k_values, cv_scores, marker="o")
plt.xlabel("Número de vizinhos (k)")
plt.ylabel("Acurácia média (5-fold CV)")
plt.title("Curva de validação para KNN")
plt.grid(True)
plt.show()

from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

# Treinar modelo final
knn = KNeighborsClassifier(n_neighbors=best_k, n_jobs=-1)
knn.fit(X_train, y_train)

# Previsões no teste
y_pred = knn.predict(X_test)

# Avaliação
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia no teste (k={best_k}): {acc:.2%}")

# Matriz de confusão
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, cmap="Blues")
plt.title(f"Matriz de confusão - KNN (k={best_k})")
plt.show()

# Relatório de classificação
print("Relatório de classificação (precision, recall, f1):")
print(classification_report(y_test, y_pred))


### Etapa 6 - Scoring nas novas localidades

In [ ]:
# Copiar sugestões
df_sug = df_sugestoes.copy()

# Criar identificador único da localidade sugerida
df_sug["localidade"] = (
    df_sug["escopo"].astype(str) + "-" + df_sug["estado"].astype(str).str.upper()
)

# Mapear estado -> macro-região (mesmo dicionário da etapa anterior)
mapa_uf_regiao = {
    "AC":"NORTE","AP":"NORTE","AM":"NORTE","PA":"NORTE","RO":"NORTE","RR":"NORTE","TO":"NORTE",
    "AL":"NORDESTE","BA":"NORDESTE","CE":"NORDESTE","MA":"NORDESTE","PB":"NORDESTE","PE":"NORDESTE","PI":"NORDESTE","RN":"NORDESTE","SE":"NORDESTE",
    "DF":"CENTRO-OESTE","GO":"CENTRO-OESTE","MT":"CENTRO-OESTE","MS":"CENTRO-OESTE",
    "ES":"SUDESTE","MG":"SUDESTE","RJ":"SUDESTE","SP":"SUDESTE",
    "PR":"SUL","RS":"SUL","SC":"SUL"
}
df_sug["Dim_Lojas.Regiao"] = df_sug["estado"].str.upper().map(mapa_uf_regiao)

# Padronizar nome da região chilli
df_sug["Dim_Lojas.REGIAO_CHILLI"] = df_sug["regiao_chilli"].astype(str).str.upper().str.strip()

# Todas as categorias conhecidas do treino
todas_categorias = sorted(df_modelo["Categoria_Combo"].dropna().unique().tolist())

# Produto cartesiano: cada local sugerido × todas as categorias
df_grid = (
    df_sug[["localidade", "Dim_Lojas.Regiao", "Dim_Lojas.REGIAO_CHILLI"]]
        .merge(pd.DataFrame({"Categoria_Combo": todas_categorias}), how="cross")
)

print("Grid de scoring:", df_grid.shape)
display(df_grid.head())

# One-hot encoding
X_new = pd.get_dummies(
    df_grid[["localidade","Categoria_Combo","Dim_Lojas.Regiao","Dim_Lojas.REGIAO_CHILLI"]],
    drop_first=True
)

# Reindex para garantir MESMAS colunas do treino
X_new = X_new.reindex(columns=X_train.columns, fill_value=0)

print("Shape X_new (alinhado):", X_new.shape)


# Predição com modelo treinado (knn já definido)
pred_label = knn.predict(X_new)
probas = knn.predict_proba(X_new)

# Montar dataframe com previsões
df_pred = df_grid.copy()
df_pred["pred_label"] = pred_label

# Adicionar colunas de probabilidade por classe
for i, cls in enumerate(knn.classes_):
    df_pred[f"proba_{cls}"] = probas[:, i]

# Probabilidade de sucesso = probabilidade de ser "bom"
df_pred["pct_sucesso"] = df_pred["proba_bom"] if "proba_bom" in df_pred.columns else np.nan

display(df_pred.head())

# Ordenar por localidade sugerida e % de sucesso
df_resultado = (
    df_pred.sort_values(["localidade","pct_sucesso"], ascending=[True, False])
             .loc[:, ["localidade","Categoria_Combo","pred_label","pct_sucesso",
                      "proba_bom","proba_mediano","proba_ruim"]]
)

display(df_resultado.head(20))

# Top 5 categorias por localidade sugerida
topN = (
    df_resultado.groupby("localidade")
                .head(5)
)

display(topN)


### Etapa 7 - Exportar Resultados

In [ ]:
# Exportar resultado completo
caminho_completo = "../../database/dataset gerado/resultados_completos.csv"
df_resultado.to_csv(caminho_completo, index=False, encoding="utf-8-sig")

print(f"Arquivo salvo: {caminho_completo}")

# Exportar Top 5 categorias por local sugerido
caminho_topN = "../../database/dataset gerado/resultados_top5_por_local.csv"
topN.to_csv(caminho_topN, index=False, encoding="utf-8-sig")

print(f"Arquivo salvo: {caminho_topN}")
